In [1]:
import pandas as pd
import numpy as np
import requests
from citipy import citipy
from config import weather_api_key

In [2]:
#3. Create a new set of 2,000 random lats and longs
lats = np.random.uniform(-90, 90, 2000)
longs = np.random.uniform(-90, 90, 2000)

coordinates = zip(lats,longs)

coordinates

In [3]:
#4. Get nearest city to each of these coords
cities = []

for i in coordinates:
    nearest = citipy.nearest_city(i[0], i[1])
    city_name = nearest.city_name
    country_code = nearest.country_code
    if (city_name, country_code) not in cities:
        cities.append((city_name, country_code))
        
cities

[('naryan-mar', 'ru'),
 ('ushuaia', 'ar'),
 ('kabompo', 'zm'),
 ('saint-philippe', 're'),
 ('slatina-timis', 'ro'),
 ('clyde river', 'ca'),
 ('san antonio', 'cl'),
 ('bredasdorp', 'za'),
 ('walvis bay', 'na'),
 ('hermanus', 'za'),
 ('adrar', 'dz'),
 ('port elizabeth', 'za'),
 ('jamestown', 'sh'),
 ('rio grande', 'br'),
 ('tasiilaq', 'gl'),
 ('umm lajj', 'sa'),
 ('qaanaaq', 'gl'),
 ('letterkenny', 'ie'),
 ('bidar', 'in'),
 ('chicama', 'pe'),
 ('altayskoye', 'ru'),
 ('henties bay', 'na'),
 ('llata', 'pe'),
 ('beira', 'mz'),
 ('hualmay', 'pe'),
 ('carnarvon', 'au'),
 ('bambous virieux', 'mu'),
 ('belushya guba', 'ru'),
 ('shache', 'cn'),
 ('paptalaya', 'hn'),
 ('busselton', 'au'),
 ('asosa', 'et'),
 ('antofagasta', 'cl'),
 ('onega', 'ru'),
 ('east london', 'za'),
 ('tsihombe', 'mg'),
 ('port alfred', 'za'),
 ('sao jose da coroa grande', 'br'),
 ('kutum', 'sd'),
 ('luangwa', 'zm'),
 ('dikson', 'ru'),
 ('mar del plata', 'ar'),
 ('tazovskiy', 'ru'),
 ('marawi', 'sd'),
 ('tchaourou', 'bj'),
 

In [4]:
# a lot of cities have the same name in different countries, by checking the country the odds are greater that we end up with a larger selection of cities
# cities is now a list of tuples

len(cities)

780

In [5]:
#5. API Calling with OpenWeatherMap with our list of cities - DO NOT PRINT API KEY in CELL OUTPUT
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
test = requests.get(url+'&q=paris,us')
test

<Response [200]>

In [7]:
test.json()

{'coord': {'lon': -95.5555, 'lat': 33.6609},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 64.4,
  'feels_like': 62.13,
  'temp_min': 64.4,
  'temp_max': 64.4,
  'pressure': 1028,
  'humidity': 34},
 'visibility': 10000,
 'wind': {'speed': 17.27, 'deg': 170, 'gust': 20.71},
 'clouds': {'all': 90},
 'dt': 1617470142,
 'sys': {'type': 1,
  'id': 5407,
  'country': 'US',
  'sunrise': 1617451590,
  'sunset': 1617497034},
 'timezone': -18000,
 'id': 4717560,
 'name': 'Paris',
 'cod': 200}

In [8]:
cities_data = []

failed_count = 0
for i,city in enumerate(cities):
    
    #make url for each city
    city_request = url + '&q=' + city[0].replace(" ","+") + ',' + city[1]
    
    single_city = {}
    
    try:
        city_results = requests.get(city_request).json()
        
        single_city["City"] = city_results['name']
        single_city["Country"] = city_results['sys']['country']
        single_city['Lat'] = round(city_results['coord']['lat'],2)
        single_city['Lng'] = round(city_results['coord']['lon'],2)
        single_city['Max Temp'] = city_results['main']['temp_max']
        single_city['Humidity'] = city_results['main']['humidity']
        single_city['Cloudiness'] = city_results['clouds']['all']
        single_city['Wind Speed'] = city_results['wind']['speed']
        single_city['Current Description'] = city_results['weather'][0]['description']
        
        cities_data.append(single_city)
        
    except:
        failed_count += 1
print(f"Failed to Retrieve Weather Data for {failed_count} Cities")

Failed to Retrieve Weather Data for 78 Cities


In [9]:
cities_database = pd.DataFrame(cities_data)
cities_database

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Naryan-Mar,RU,67.67,53.09,10.71,93,98,3.53,overcast clouds
1,Ushuaia,AR,-54.80,-68.30,39.20,93,90,24.16,snow
2,Kabompo,ZM,-13.59,24.20,67.89,72,30,5.28,scattered clouds
3,Saint-Philippe,RE,-21.36,55.77,78.80,83,20,4.61,light rain
4,Slatina-Timiş,RO,45.25,22.28,42.22,94,100,3.65,overcast clouds
...,...,...,...,...,...,...,...,...,...
697,Caconda,AO,-13.73,15.07,64.22,86,93,5.86,light rain
698,La Sarre,CA,48.80,-79.20,37.74,46,100,9.26,overcast clouds
699,Ust'-Koksa,RU,50.27,85.61,21.60,71,0,2.15,clear sky
700,Rāichūr,IN,16.20,77.37,90.70,17,42,7.43,scattered clouds


In [10]:
try:
    cities_database.to_csv('WeatherPy_Database.csv')
    print('File Saved Successfully')
except:
    print('Failed to Save File!')

File Saved Successfully
